# Landsat 8 OLI TIRS Reflectancia Superficie google Earth Engine

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
ee_zona = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap="ESRI")

In [5]:
# Visualizar en mapa
geometria = ee_zona.geometry()
Map.centerObject(geometria, 11)
Map.addLayer(ee_zona, {"color" : "red"}, name = "Zona")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Coleccion de Landsat 8 OLI TIRS RS Colleccion 2

In [6]:
L8_RS = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
          .filterDate('2019-01-01','2019-12-31')\
          .filterMetadata('CLOUD_COVER','less_than',20)\
          .filterBounds(ee_zona)

In [7]:
# Conteo de imagenes
count = L8_RS.size().getInfo()
print("Cantidad de imagenes L8_RS:", count)

Cantidad de imagenes L8_RS: 4


## Obtener ID, Fecha, nubosidad, Path y Row

In [8]:
# Imprimir la lista de ID Imagenes
ID_L8 = L8_RS.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(ID_L8)

['LC08_003069_20190621', 'LC08_003069_20190808', 'LC08_003069_20190824', 'LC08_003069_20190925']


In [9]:
# Crear funcion de extraer la fecha
def list_coll(coll):
    def func_ejv(im):
        return ee.String(ee.Image(im).date().format('YYYY/MM/dd')).slice(0)
    return coll.toList(coll.size(), 0).map(func_ejv)

In [10]:
fecha = list_coll(L8_RS).getInfo()
print(fecha)

['2019/06/21', '2019/08/08', '2019/08/24', '2019/09/25']


In [11]:
# Imprimir la lista Porcentaje Nubosidad
porcentaje_nube = L8_RS.reduceColumns(ee.Reducer.toList(),["CLOUD_COVER"]).get("list").getInfo()
print(porcentaje_nube)

[11.45, 1.1, 18.05, 5.46]


In [13]:
# Imprimir la lista Path y Row
WRS_PATH = L8_RS.reduceColumns(ee.Reducer.toList(),["WRS_PATH"]).get("list").getInfo()
WRS_ROW = L8_RS.reduceColumns(ee.Reducer.toList(),["WRS_ROW"]).get("list").getInfo()
print("PATH:", WRS_PATH)
print("ROW:",WRS_ROW)

PATH: [3, 3, 3, 3]
ROW: [69, 69, 69, 69]


In [15]:
# Crear un diccionario con las listas
dic = {"ID": ID_L8, "Fecha": fecha,"Porcentaje Nube": porcentaje_nube,"PATH": WRS_PATH,"ROW":WRS_ROW}

In [16]:
import pandas as pd

In [17]:
# Crear un DataFrame mediante el diccionario
df1 = pd.DataFrame(dic)
print(df1)
print("Tipo de clase: ", type(df1))

                     ID       Fecha  Porcentaje Nube  PATH  ROW
0  LC08_003069_20190621  2019/06/21            11.45     3   69
1  LC08_003069_20190808  2019/08/08             1.10     3   69
2  LC08_003069_20190824  2019/08/24            18.05     3   69
3  LC08_003069_20190925  2019/09/25             5.46     3   69
Tipo de clase:  <class 'pandas.core.frame.DataFrame'>


## Exportar tabla ID, Fecha, Porcentaje Nubosidad

In [18]:
import os

In [19]:
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [20]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [21]:
df1.to_csv("file_name_L8_Collecion2.csv") # relative position

## Seleccion imagen Landsat 8

In [25]:
"LANDSAT/LC08/C02/T1_L2" + "/" + ID_L8[1]

'LANDSAT/LC08/C02/T1_L2/LC08_003069_20190808'

In [26]:
# Importar imagen Landsat 8 RS 
image = ee.Image("LANDSAT/LC08/C02/T1_L2" + "/" + ID_L8[1])

In [27]:
# Nombre de las bandas
print(image.bandNames().getInfo())

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'SR_B6', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']


## Seleccionar Bandas

In [28]:
# Crear nombre por bandas
bandas_RS = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
bandas_TB = ["ST_B10"]

In [29]:
# Seleccionar imagenes por bandas
img_L8_RS = image.select(bandas_RS)
img_L8_TB = image.select(bandas_TB)

In [30]:
# Nombre de las bandas
print("Banda RS:",img_L8_RS.bandNames().getInfo())
print("Banda TB:",img_L8_TB.bandNames().getInfo())

Banda RS: ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
Banda TB: ['ST_B10']


## Verificar proyeccion

In [31]:
# Consultar Proyeccion
print("Proyeccion RS:", img_L8_RS.projection().getInfo())
print("Proyeccion TB:", img_L8_TB.projection().getInfo())

Proyeccion RS: {'type': 'Projection', 'crs': 'EPSG:32619', 'transform': [30, 0, 228285, 0, -30, -1323585]}
Proyeccion TB: {'type': 'Projection', 'crs': 'EPSG:32619', 'transform': [30, 0, 228285, 0, -30, -1323585]}


## Metadato landsat

In [32]:
# Nombres del Metadato
img_L8_RS.propertyNames().getInfo()

['DATA_SOURCE_ELEVATION',
 'WRS_TYPE',
 'system:id',
 'REFLECTANCE_ADD_BAND_1',
 'REFLECTANCE_ADD_BAND_2',
 'DATUM',
 'REFLECTANCE_ADD_BAND_3',
 'REFLECTANCE_ADD_BAND_4',
 'REFLECTANCE_ADD_BAND_5',
 'REFLECTANCE_ADD_BAND_6',
 'REFLECTANCE_ADD_BAND_7',
 'system:footprint',
 'REFLECTIVE_SAMPLES',
 'system:version',
 'GROUND_CONTROL_POINTS_VERSION',
 'SUN_AZIMUTH',
 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION',
 'UTM_ZONE',
 'DATE_ACQUIRED',
 'ELLIPSOID',
 'system:time_end',
 'DATA_SOURCE_PRESSURE',
 'LANDSAT_PRODUCT_ID',
 'STATION_ID',
 'TEMPERATURE_ADD_BAND_ST_B10',
 'DATA_SOURCE_REANALYSIS',
 'REFLECTANCE_MULT_BAND_7',
 'system:time_start',
 'REFLECTANCE_MULT_BAND_6',
 'L1_PROCESSING_LEVEL',
 'PROCESSING_SOFTWARE_VERSION',
 'L1_DATE_PRODUCT_GENERATED',
 'ORIENTATION',
 'REFLECTANCE_MULT_BAND_1',
 'WRS_ROW',
 'REFLECTANCE_MULT_BAND_3',
 'REFLECTANCE_MULT_BAND_2',
 'TARGET_WRS_ROW',
 'REFLECTANCE_MULT_BAND_5',
 'REFLECTANCE_MULT_BAND_4',
 'THERMAL_LINES',
 'TIRS_SSM_POSITION_STATUS',
 'GRID

In [33]:
# Extraer valor metadato distancia del sol a tierra
d = img_L8_RS.get("EARTH_SUN_DISTANCE").getInfo()
d

1.0140333

In [35]:
# Extraer angulo Elevacion Solar
SUN_ELEVATION = img_L8_RS.get("SUN_ELEVATION").getInfo()
SUN_ELEVATION

48.29906763

In [36]:
import math

In [37]:
SUN_ELEVATION_R = SUN_ELEVATION*math.pi/180
SUN_ELEVATION_R

0.8429777557869145

## Visualizar Landsat

In [38]:
# Simbologia estala 1 - 65455
viz_RS = {
    'bands': ['SR_B6', 'SR_B5', 'SR_B4'],
    'min': 100,
    'max': 25000,
    'gamma': 0.5
}

In [39]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [40]:
# Convertir en Geometria
geometria = ee_zona.geometry()
Map.centerObject(geometria, 8)

In [41]:
# Visualizar ESCALADO 1 - 65455
Map.addLayer(img_L8_RS, viz_RS, name = "Landsat8 RS")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Factor de escala Reflectancia Superficie

In [42]:
# Escalar la imagen Landsat 8 RS 0 - 1
img_L8_RS_Es = img_L8_RS.multiply(0.0000275).add(-0.2)

In [43]:
# Simbologia estala 0 - 1
viz_es = {
    'bands': ['SR_B6', 'SR_B5', 'SR_B4'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [44]:
# Visualizar ESCALADO 0 - 1
Map.addLayer(img_L8_RS_Es, viz_es, name = "Landsat8 RS Escalado")
Map

Map(bottom=70898.09094238281, center=[-13.768973757679376, -71.38581507985927], controls=(WidgetControl(option…

## Factor de escala Temperatura Brillo

In [45]:
# Escalar la imagen Landsat 8 Temperatura de superficial
img_L8_TB_Es = img_L8_TB.multiply(0.00341802).add(149)

In [46]:
# Escalar la imagen Landsat 8 TB convertir Kelvin a Celsius
img_L8_TB_Es = img_L8_TB_Es.subtract(273.15)

In [47]:
# Crear su simbologia TB
viz_tb = {
    'palette': ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                'ff0000', 'de0101', 'c21301', 'a71001', '911003'],
    'min': 25,
    'max': 30
}

In [48]:
# Visualizacion temperatura brillo
Map.addLayer(img_L8_TB_Es, viz_tb, name = "TB Celsius")
Map

Map(bottom=35474.2727355957, center=[-13.100879969526476, -70.21087646484376], controls=(WidgetControl(options…

## Reproyectar y recortar segun zona

In [53]:
# Reproyectar segun zona de estudio
img_L8_TB_Es = img_L8_TB_Es.reproject(crs="EPSG:32719", scale=30)
img_L8_RS_Es = img_L8_RS_Es.reproject(crs="EPSG:32719", scale=30)

In [54]:
# Recortar RS y TB
img_L8_TB_Es_clip = img_L8_TB_Es.clip(ee_zona)
img_L8_RS_Es_clip = img_L8_RS_Es.clip(ee_zona)

In [55]:
# Consultar Proyeccion
print("Proyeccion TB:",img_L8_TB_Es_clip.projection().getInfo())
print("Proyeccion RS:",img_L8_RS_Es_clip.projection().getInfo())

Proyeccion TB: {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion RS: {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}


In [56]:
img_L8_RS_Es_clip.bandNames().getInfo()

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']

In [57]:
# Visualizacion TB y RS
Map.addLayer(img_L8_TB_Es_clip, viz_tb,name = "L8 TB clip")
Map.addLayer(img_L8_RS_Es_clip, viz_es,name = "L8 RS clip")
Map

Map(bottom=35471.0, center=[-13.08482887454732, -70.88653564453126], controls=(WidgetControl(options=['positio…

## Exportar Imagen calibrado

In [58]:
os.getcwd()

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [59]:
# Exportar imagen Temperatura de brillo TB10
geemap.ee_export_image(img_L8_TB_Es_clip, filename="L8_TB_es_clip_B10.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_TB_es_clip_B10.tif


In [60]:
# Exportar imagen Reflectancia superficie B654
geemap.ee_export_image(img_L8_RS_Es_clip.select(['SR_B6', 'SR_B5', 'SR_B4']), filename="L8_RS_Es_clip_B654_coleccion2.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_RS_Es_clip_B654_coleccion2.tif


In [61]:
# Exportar en google drive
geemap.ee_export_image_to_drive(img_L8_RS_Es_clip, description='L8_RS_Es_clip_collecion2', folder='GEE_Python', region=geometria, scale=30)

Exporting L8_RS_Es_clip_collecion2 ...
